<a href="https://colab.research.google.com/github/anirudh-tatavarthi/Twittorials/blob/master/Finding_URL_Parameters_through_Log_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Not all URL parameters show up in search console or the sitemaps. By studying the logs, you can find these parameters to locate duplicate content

### This script will allow you to identify the many URL parameters present in your log files, and the number of times they appear on unique URLs

In [1]:
#This block of code installs the necessary packages
%%capture
import csv
!pip install lars
from lars.apache import ApacheSource
import plotly.graph_objects as go

**Step 1:** When this block runs, you will be prompted to choose a file. Select a text file containing the logs.

In [ ]:
from google.colab import files

#prompts user to choose a file
uploaded = files.upload()

#reads the name of the file
for fn in uploaded.keys():
  file_name = fn

**Step 2:** This next block of code goes through the logs and gathers information on all of the parameters.

At the end, you will be able to view a bar graph in order to visualize the data, as well as a download to view the data in an excel file

In [ ]:
import csv
from lars.apache import ApacheSource

#create sets storing parameters and urls
paramset = dict()
urlset = set()

#open the file and identify the parameters with Apache Source
with open(file_name) as f:
    with ApacheSource(f) as source:
        for row in source:
          if(len(row.request.url.query_str)>0):
             parameter = "?" + row.request.url.query_str
             url = row.request.url.path_str+parameter
             while (parameter.find('&')!=-1):
                parameter.find('=')
                sub = parameter[parameter.find('&')+1:parameter.find('=')]
                if(len(sub)>0):
                  if(sub not in paramset.keys()):
                    paramset[sub]=0
                parameter = parameter[parameter.find('=')+1:len(parameter)]
             parameter = "?" + row.request.url.query_str
             if((url in urlset) ==False):
              while (parameter.find('&')!=-1):
                  parameter.find('=')
                  sub = parameter[parameter.find('&')+1:parameter.find('=')]
                  if(len(sub)>0): 
                    paramset[sub]=paramset.get(sub)+1
                  parameter = parameter[parameter.find('=')+1:len(parameter)]
              urlset.add(url)
        sort_orders = sorted(paramset.items(), key=lambda x: x[1])
        paramset=dict(sort_orders)   
        for k in paramset.keys():
          output = "[Parameter: " + k + " , count: " + str(paramset[k]) + " ]"
          with open('output.csv', 'a', newline='') as f:
                thewriter = csv.writer(f)
                thewriter.writerow([output])

files.download('output.csv')

#sorts the dictionary of parameters from least appeared to most
sort_orders = sorted(paramset.items(), key=lambda x: x[1], reverse=False)

key_list = list(paramset.keys()) 
val_list = list(paramset.values()) 

#creates a horizontal bar graph with the data
fig = go.Figure([go.Bar(x=val_list, y=key_list, orientation='h')])
fig.show()

#completion message
print("Done!")
